In [9]:
from __future__ import print_function
import argparse

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.datasets import MNIST
from torch.autograd import Variable

In [17]:
# settings
batch_size = 128
batch_size_test = 1000
n_epochs = 10
learning_rate = 0.01
momentum = 0.9
use_cuda = True
rnd_seed = 1
log_interval = 10

In [18]:
# CapsNet
from capsnet import CapsNetWithReconstruction, CapsNet, ReconstructionNet

In [19]:
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.Pad(2), transforms.RandomCrop(28),
                           transforms.ToTensor()
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)

In [25]:
for batch_idx, (data, target) in enumerate(train_loader):
    data, target = Variable(data), Variable(target)
    conv1 = nn.Conv2d(1, 10, kernel_size=5)
    out = conv1(data)
    out1 = F.max_pool2d(out,2)
    print(data.shape)
    print(out.shape)
    print(out1.shape)
    if batch_idx>0:
        break

torch.Size([128, 1, 28, 28])
torch.Size([128, 10, 24, 24])
torch.Size([128, 10, 12, 12])
torch.Size([128, 1, 28, 28])
torch.Size([128, 10, 24, 24])
torch.Size([128, 10, 12, 12])
